# Kafka Producer
**Ejemplo de cliente Productor de Kafka**
Es una plantilla de programa en la que se encuentran porciones de código que deben ser rellenadas. Todas esas porciones están indicadas con texto.

Este programa genera unos datos con la librería *Faker* y envía los datos a un servidor Kafka.

Presta atención a los siguientes parámetros de la instalación:
- Servidores o servidores que tienes disponibles así como sus puertos
- Topic en el que quieres almacenar los datos
- Valor de la clave a utilizar en los registros
- En el fichero *Producer.log* se guardan toda la información. 

Ten disponible el Offset Explorer para confirmar los valores


### Importando módulos necesarios

In [1]:
from kafka import KafkaProducer
from faker import Faker
from json import dumps, loads 
import time
import logging
import sys
import random

### Configuramos Logging y Faker

In [2]:
#Logging
logging.basicConfig(format='%(asctime)s %(message)s',
                    datefmt='%d-%m-%Y %H:%M:%S',
                    filename='producer.log',
                    filemode='w')

logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [3]:
#Faker
fake=Faker('es_ES')

### Configuración básica

Tendrás que especificar el Topic, la clave y el/los servidores

In [4]:
#Configuración del entorno Kafka
topic_name = 'my-topic'
key = 'my-key'
bootstrap_servers = ['localhost:9092', 'localhost:9093']
logging.info('Arrancando...')

### Configuración del Productor

Los serializadores son funcinoes llamables que ayudarán a enviar, o interpretar, los datos del servidor.

En esta plantilla hay dos, uno para la clave y otro para el dato. Tendrás que configurar el de la clave, que es, simplemente, una cadena.

In [5]:
#Definiendo el conector
def connect_kafka_producer(bootstrap_servers):
    producer = None
    try:
        producer = KafkaProducer(bootstrap_servers=bootstrap_servers,
                                 key_serializer=str.encode(),
                                 value_serializer=lambda x: dumps(x).encode('utf-8'))
    except Exception as ex:
        logging.error('Exception while connecting producer')
        logging.error(ex)
    finally:
        return producer

### Definiendo la publicación del Mensaje en Kafka

In [6]:
#Definiendo la publicación del mensaje
def publish_message(producer, topic, key, value):
    try:
        producer.send(topic, key=key, value=value)
        producer.flush()
        logging.debug('Publicación del Mensaje OK')
    except Exception as ex:
        logging.error(value)
        logging.error('Excepción al publicar el mensaje')
        logging.error(ex)
    finally:
        return

### Creando datos de usuario y publicando

Sustituye las cadenas de *?????* por sus valores correspondientes

In [8]:
producer = connect_kafka_producer(bootstrap_servers)
logging.info("Conexión a broker: "+str(producer))
for i in range(30):
    data={
       'user_id': fake.random_int(min=20000, max=100000),
       'user_name':fake.name(),
       'user_address':fake.street_address() + ' | ' + fake.city() + ' | ' + fake.country_code(),
       'platform': random.choice(['Mobile', 'Laptop', 'Tablet']),
       'signup_at': str(fake.date_time_this_month())    
    }
    publish_message(producer, topic_name, key, data)
    logging.info(data)
    time.sleep(1)
